In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

train = pd.read_csv("train_preprocessed.csv")

In this nb we'll play around with positive examples, trying to infer the possible types of questions available. 

First, let's drop all the negative examples. 

In [2]:
train.columns

Index(['Unnamed: 0', 'id', 'qid1', 'qid2', 'question1', 'question2',
       'is_duplicate'],
      dtype='object')

In [3]:
train.head()

,Unnamed: 0,id,qid1,qid2,question1,question2,is_duplicate
0,0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0
1,1,1,3,4,what is the story of kohinoor (koh-i-noor) dia...,what would happen if the indian government sto...,0
2,2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0
3,4,4,9,10,"which one dissolve in water quikly sugar, salt...",which fish would survive in salt water?,0
4,5,5,11,12,astrology: i am a capricorn sun cap moon and c...,"i'm a triple capricorn (sun, moon and ascendan...",1


In [10]:
train = train.loc[ train.is_duplicate == 1 , lambda df: ['question1', 'question2'] ] 

We'll now change the questions to be lists of words, and remove all words below some frequency. This should leave only the interesting words (the correct thing to do is probably use them as well as word2vec, so that some of the meaning remains)